In [1]:
import nltk
import pandas as pd
import re
regex = re.compile('[^A-Za-zÀ-ÿ]')

import seaborn as sns
import numpy as np
import itertools
from tqdm import tqdm
from functools import reduce
import matplotlib.pyplot as plt
from collections import Counter
from nltk.corpus import wordnet

from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer() 

import spacy
sp = spacy.load('en_core_web_sm') # dans terminal : python -m spacy download en_core_web_sm

import itertools
import collections
import pickle

from typing import List
import wordninja

from spellchecker import SpellChecker
spell = SpellChecker()

from nltk.corpus import stopwords
pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*') # POUR REMOVE LE STOPWORD

In [2]:
#DATA_TRAIN_POS_PATH = './Datasets/twitter-datasets/train_pos_full.txt'  
#DATA_TRAIN_NEG_PATH = './Datasets/twitter-datasets/train_neg_full.txt'

DATA_TRAIN_POS_PATH = './Datasets/twitter-datasets/pos_train.txt'  
DATA_TRAIN_NEG_PATH = './Datasets/twitter-datasets/neg_train.txt'

DATA_TEST_PATH = './Datasets/twitter-datasets/test_data.txt'

In [3]:
# load the data files = list with each line being a tweet
pos_data = open(DATA_TRAIN_POS_PATH, "r").read().splitlines()
neg_data = open(DATA_TRAIN_NEG_PATH, "r").read().splitlines()

test_data = open(DATA_TEST_PATH, "r").read().splitlines()

In [4]:
len(pos_data)

100000

In [5]:
len(test_data)

10000

In [6]:
def clean_tweet(twt):
    
    # get words in hashtags and removes smiley
    tweet = [' '.join(wordninja.split(word)) if word.startswith("#") \
             else (smileys.get(word) if word in smileys else word) for word in twt.split()]
    tweet=' '.join(tweet)
    
    #print("\n 1!!!!",tweet)
    
    # expand contractions like don't -> do not    
    tweet = expandContractions(tweet, c_re=c_re)
    
    #print("\n 2!!!!",tweet)

    # correct the elongated words
    tweet = [remove_consecutive_dups(word) if len(spell.unknown([remove_consecutive_dups(word)])) == 0 \
            else (spell.correction(remove_consecutive_dups(word))) for word in tweet.split()]

    tweet = ' '.join(tweet)
    
    #print("\n 3!!!!",tweet)
    
    return tweet

In [7]:
smileys = {
    ':-)': '',
    ':)': '',
    ':-3': '',
    ':3': '',
    ':-]': '',
    ':]': '',
    ':->': '',
    ':>': '',
    ':-}': '',
    ':}': '',
    '=)': '',
    '=]': '',
    ':P': '',
    ':p': '',
    ':-P': '',
    ':D': '',
    ':-D': '',
    'xD': '',
    'XD': '',
    '=D': '',
    '=3': '',
    ':-(': '',
    ':(': '',
    ':-[': '',
    ':c': '',
    ':-c': '',
    ':[': '',
    ':<': '',
    ':-<': '',
    ':-{': '',
    ':{': '',
    ':@': '',
    ":'(": '',
    ":-'(": '',
    ":-')": '',
    ":')": '',
    ':-*': '',
    ':*': '',
    ':x': '',
    'xx': '',
    'xxx': '',
    'xo': '',
    'xoxo': '',
    'xoxoxo': '',
    'xoxoxox': '',
    '<3': '',
    ':-o': '',
    ':o': '',
    ':O': '',
    ':-O': '',
    ';)': '',
    ';-)': '',
}

cList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "can not",
  "can't've": "can not have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "i'd": "I would",
  "i'd've": "I would have",
  "i'll": "I will",
  "i'll've": "I will have",
  "i'm": "I am",
  "i've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

In [8]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [9]:
def remove_consecutive_dups(s):
    return re.sub(r'(.)\1+', r'\1\1', s)

In [10]:
def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

In [11]:
def parse_smileys(tweet: List[str]) -> List[str]:
    return [smileys.get(word, word) for word in tweet]

In [12]:
def remove_ids(test_data):
    return [twt.split(',', 1)[-1] for twt in test_data]

In [13]:
def preprocessing(data):
    
    # remove empty tweets
    data = [twt for twt in data if not (twt.isspace() or twt == np.nan or not twt)]
    #print("rmv empty:",data)
    
    # remove tags
    data = [twt.replace('<user>', '') for twt in data]
    data = [twt.replace('<url>', '') for twt in data]
    #print("\n rmv tags:",data)
    
    # convert into lower cases
    data = [twt.lower() for twt in data] 
    #print("\n lower cases:",data)
    
    # take from word from hashtag, removes smileys and corrects elongated words
    data = [clean_tweet(twt) for twt in data]
    #print("\n\ #:)yooo",data)
    
    # remove all non-alphabetical characters 
    data = [regex.sub(' ', twt) for twt in data]
    #print("\n rmv symbols",data)
    
    # remove empty tweets
    data = [twt for twt in data if not (twt.isspace() or twt == np.nan or not twt)]
    #print("\n rmv empty",data)
    
    # substitute multiple whitespace with single whitespace 
    data = [' '.join(twt.split()) for twt in data]
    #print("\n rmv spaces",data)
    
    # remove stopwords
    data = [pattern.sub('', twt) for twt in data]
    #print("\n rmv stopwords",data)
    
    # remove words that are < 2 letters
    data = [[token for token in twt.split() if len(token) > 2] for twt in data]
    data = [' '.join(twt) for twt in data]
    #print("\n rmv small words",data)
    
    # tokenize
    data = [sp(twt) for twt in data]
    #print("\n toktok",data)
    
    # get POS-tag of words of each tweet
    data = [[(token.text, token.tag_) for token in twt]  for twt in data]
    #print("POS-tag",data)
    
    # transform POS tag + lemmatize
    data = [[lemmatizer.lemmatize(twt[i][0], get_wordnet_pos(twt[i][1])) for i in range(len(twt))] for twt in data]
    #print("P-t -> lemmatize",data)
    
    # back to tweet format
    data = [' '.join(twt) for twt in data]
    #print("twt format",data)
    
    # remove words that are < 2 letters
    data = [[token for token in twt.split() if len(token) > 2] for twt in data]
    data = [' '.join(twt) for twt in data]
    #print("\n rmv small words",data)
    
    return data 

In [14]:
test_data = remove_ids(test_data)
test_data

['sea doo pro sea scooter ( sports with the portable sea-doo seascootersave air , stay longer in the water and ... <url>',
 "<user> shucks well i work all week so now i can't come cheer you on ! oh and put those batteries in your calculator ! ! !",
 'i cant stay away from bug thats my baby',
 "<user> no ma'am ! ! ! lol im perfectly fine and not contagious anymore lmao",
 'whenever i fall asleep watching the tv , i always wake up with a headache',
 "<user> he needs to get rid of that thing ! it scares me lol but he don't need a car either . he needs drivers ed again .",
 'its whatever . in a terrible mood ( (',
 "yesss ! rt <user> <user> thanks jordan , i love you and i'm gonna call you later !",
 'my friend <user> text me to check up on me last night .',
 '<user> #followback please . when will ur #unitytour come to europe and sweden ? ?',
 "watch some of y'all dumb asses get lock up today #happy420",
 'obsessed with #phasell <user> you killed it ! ! ! best album ever love yew roycee ! 

In [15]:
%%time
result_test = preprocessing(test_data)

CPU times: user 40min 2s, sys: 2min 15s, total: 42min 18s
Wall time: 39min 41s


In [23]:
len(result_test)

10000

In [16]:
with open("res_test.txt", "w") as output:
    output.write(str(result_test))

In [18]:
testFile=open('result_test.txt','w')

for element in result_test:
    testFile.write(element)
    testFile.write('\n')
testFile.close()

In [19]:
# remove tweets duplicates
result_test_ = list(dict.fromkeys(result_test))

In [20]:
with open("res_test_nd.txt", "w") as output:
    output.write(str(result_test_))

In [21]:
testFile_=open('result_test_nd.txt','w')

for element in result_test_:
    testFile_.write(element)
    testFile_.write('\n')
testFile_.close()

In [ ]:
%%time
result_pos_train = preprocessing(pos_data)
result_neg_train = preprocessing(neg_data)

In [ ]:
with open("res_pos_train.txt", "w") as output:
    output.write(str(result_pos_train))
with open("res_neg_train.txt", "w") as output:
    output.write(str(result_pos_train))

In [ ]:
PosFile=open('result_pos_train.txt','w')

for element in result_pos_train:
    PosFile.write(element)
    PosFile.write('\n')
PosFile.close()

In [ ]:
NegFile=open('result_neg_train.txt','w')

for element in result_neg_train:
    NegFile.write(element)
    NegFile.write('\n')
NegFile.close()

In [ ]:
# remove tweets duplicates
result_pos_train_ = list(dict.fromkeys(result_pos_train))
result_neg_train_ = list(dict.fromkeys(result_neg_train))

In [ ]:
with open("resPtrain_nd.txt", "w") as output:
    output.write(str(result_pos_train_))
with open("resNtrain_nd.txt", "w") as output:
    output.write(str(result_neg_train_))

In [ ]:
PosFile=open('resultPtrain_nd.txt','w')

for element in result_pos_train_:
    PosFile.write(element)
    PosFile.write('\n')
PosFile.close()

In [ ]:
NegFile=open('resultNtrain_nd.txt','w')

for element in result_neg_train_:
    NegFile.write(element)
    NegFile.write('\n')
NegFile.close()